In [1]:
import json

import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')


In [3]:
# !ls

In [14]:
def preprocess(input, only_char=True, lower=True, stop_remove=True, stemming=False):
    input = re.sub(r'[^\x00-\x7F]+',' ', input)
    if lower: input = input.lower()
    if only_char:
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(input)
        input = ' '.join(tokens)
    tokens = word_tokenize(input)
    if stop_remove:
        tokens = [w for w in tokens if not w in stopwords.words('english')]

    # also remove one-length word
    tokens = [w for w in tokens if len(w) > 1]
    return " ".join(tokens)

In [15]:
import pandas as pd
columns = ['RECOMMENDATION', 'comments', 'CONFIDENCE', 'Layer 1', 'Layer 1 N', 'Layer 2', 'Layer 2 N', 'Layer 3',
       'Layer 3 N']
Treviews = pd.read_csv('Test_Reviews.csv', usecols=columns)
Treviews_l = Treviews[Treviews['Layer 1 N'] < 2]
TRreviews = pd.read_csv('Train_Reviews.csv', usecols=columns)
TRreviews_l = TRreviews[TRreviews['Layer 1 N'] < 2]


In [17]:
len(Treviews), len(TRreviews)

(81, 830)

In [18]:
len(Treviews_l), len(TRreviews_l)

(24, 235)

In [26]:
with open('reviews_tst_l', 'w') as f:
    for i in range(len(Treviews_l)):
        rev_txt = Treviews_l.iloc[i]['comments']
        f.write(preprocess(rev_txt) + '\n')
    for i in range(len(TRreviews_l)):
        rev_txt = TRreviews_l.iloc[i]['comments']
        f.write(preprocess(rev_txt) + '\n')
    
    

In [20]:
# rev1 = json.load(open('R2.json'))['review']
# rev2 = json.load(open('R1.json'))['review']

In [21]:
# rev1, rev2 =  preprocess(rev1), preprocess(rev2)


In [22]:
# print(rev1)

In [23]:
# print(rev2)

In [24]:
# import os
# reviews = []
# for paper in os.listdir('./ScoreData'):
#     print(paper)
#     rev_data = json.load(open('./ScoreData/' + paper + '/reviews.json'))
#     for r in rev_data['reviews']:
#         reviews.append(preprocess(r['comments']))

In [25]:
# with open('topic_tst.txt', 'w') as f:
#     for review in reviews:
#         f.write(review + '\n')

In [6]:
from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.utils.data_preparation import TextHandler
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file
from contextualized_topic_models.datasets.dataset import CTMDataset

handler = TextHandler("reviews_tst_l")
handler.prepare() # create vocabulary and training data

# generate BERT data
training_bert = bert_embeddings_from_file("reviews_tst_l", "./roberta-large-nli-mean-tokens")#"../rev_sig/codes/models/scibert_scivocab_uncased")
print(training_bert.shape)

training_dataset = CTMDataset(handler.bow, training_bert, handler.idx2token)

ctm = CTM(input_size=len(handler.vocab), bert_input_size=1024, num_epochs=1000, inference_type="combined", n_components=50)

ctm.fit(training_dataset) # run the model

/home2/tirthankar/miniconda3_1/envs/rajeev3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home2/tirthankar/miniconda3_1/envs/rajeev3/lib/python3.7/site-packages/contextualized_topic_models/utils/data_preparation.py:161: Warning: The vocab you are using has more than 2000 words, reconstructing high-dimensional vectors requiressignificantly more training epochs and training samples. Consider reducing the number of vocabulary items. See https://github.com/MilaNLProc/contextualized-topic-models#preprocessing and https://github.com/MilaNLProc/contextualized-topic-models#tldr
  "and https://github.com/MilaNLProc/contextualized-topic-models#tldr", Warning)



(259, 1024)
Settings: 
                   N Components: 50
                   Topic Prior Mean: 0.0
                   Topic Prior Variance: 0.98
                   Model Type: prodLDA
                   Hidden Sizes: (100, 100)
                   Activation: softplus
                   Dropout: 0.2
                   Learn Priors: True
                   Learning Rate: 0.002
                   Momentum: 0.99
                   Reduce On Plateau: False
                   Save Dir: None


/home2/tirthankar/miniconda3_1/envs/rajeev3/lib/python3.7/site-packages/contextualized_topic_models/models/ctm.py:47: DeprecationWarning: Direct call to CTM is deprecated and will be removed in version 2, use CombinedTM or ZeroShotTM
  warnings.warn("Direct call to CTM is deprecated and will be removed in version 2, use CombinedTM or ZeroShotTM", DeprecationWarning)


Epoch: [1/1000]	Samples: [259/259000]	Train Loss: 892.3521563555744	Time: 0:00:04.993397
Epoch: [2/1000]	Samples: [518/259000]	Train Loss: 889.2138992368484	Time: 0:00:05.073728
Epoch: [3/1000]	Samples: [777/259000]	Train Loss: 887.608136386945	Time: 0:00:05.256637
Epoch: [4/1000]	Samples: [1036/259000]	Train Loss: 886.4355242519305	Time: 0:00:04.989182
Epoch: [5/1000]	Samples: [1295/259000]	Train Loss: 884.0136303993726	Time: 0:00:04.760300
Epoch: [6/1000]	Samples: [1554/259000]	Train Loss: 887.217014622044	Time: 0:00:04.835326
Epoch: [7/1000]	Samples: [1813/259000]	Train Loss: 885.8762273015203	Time: 0:00:04.937217
Epoch: [8/1000]	Samples: [2072/259000]	Train Loss: 884.650109721887	Time: 0:00:04.800740
Epoch: [9/1000]	Samples: [2331/259000]	Train Loss: 884.087184596706	Time: 0:00:04.863586
Epoch: [10/1000]	Samples: [2590/259000]	Train Loss: 880.7504392645994	Time: 0:00:04.809836
Epoch: [11/1000]	Samples: [2849/259000]	Train Loss: 880.8567950274494	Time: 0:00:04.940811
Epoch: [12/1000

Epoch: [91/1000]	Samples: [23569/259000]	Train Loss: 811.3590644041084	Time: 0:00:06.413091
Epoch: [92/1000]	Samples: [23828/259000]	Train Loss: 810.3361222550676	Time: 0:00:06.410732
Epoch: [93/1000]	Samples: [24087/259000]	Train Loss: 814.8144182477679	Time: 0:00:06.324388
Epoch: [94/1000]	Samples: [24346/259000]	Train Loss: 812.3115724541506	Time: 0:00:06.353556
Epoch: [95/1000]	Samples: [24605/259000]	Train Loss: 813.1506451345318	Time: 0:00:06.338354
Epoch: [96/1000]	Samples: [24864/259000]	Train Loss: 809.3082534613296	Time: 0:00:06.223201
Epoch: [97/1000]	Samples: [25123/259000]	Train Loss: 806.8359488115348	Time: 0:00:06.586632
Epoch: [98/1000]	Samples: [25382/259000]	Train Loss: 810.2160041249397	Time: 0:00:06.387899
Epoch: [99/1000]	Samples: [25641/259000]	Train Loss: 808.2517881651182	Time: 0:00:06.326554
Epoch: [100/1000]	Samples: [25900/259000]	Train Loss: 810.3109295819256	Time: 0:00:06.479745
Epoch: [101/1000]	Samples: [26159/259000]	Train Loss: 807.5959086178813	Time: 0

...

Epoch: [622/1000]	Samples: [161098/259000]	Train Loss: 728.7263230725144	Time: 0:00:05.368607
Epoch: [623/1000]	Samples: [161357/259000]	Train Loss: 728.9892455583373	Time: 0:00:05.373966
Epoch: [624/1000]	Samples: [161616/259000]	Train Loss: 725.0665344473938	Time: 0:00:05.433064
Epoch: [625/1000]	Samples: [161875/259000]	Train Loss: 719.5263737858953	Time: 0:00:05.289619
Epoch: [626/1000]	Samples: [162134/259000]	Train Loss: 722.3139332657155	Time: 0:00:05.270239
Epoch: [627/1000]	Samples: [162393/259000]	Train Loss: 734.2395325885316	Time: 0:00:05.504879
Epoch: [628/1000]	Samples: [162652/259000]	Train Loss: 728.8670932372104	Time: 0:00:05.307357
Epoch: [629/1000]	Samples: [162911/259000]	Train Loss: 720.3016211314551	Time: 0:00:05.358367
Epoch: [630/1000]	Samples: [163170/259000]	Train Loss: 724.3255440848214	Time: 0:00:05.231309
Epoch: [631/1000]	Samples: [163429/259000]	Train Loss: 724.8172225657577	Time: 0:00:05.299260
Epoch: [632/1000]	Samples: [163688/259000]	Train Loss: 726.2

Epoch: [710/1000]	Samples: [183890/259000]	Train Loss: 729.572455093207	Time: 0:00:05.386126
Epoch: [711/1000]	Samples: [184149/259000]	Train Loss: 721.452229974813	Time: 0:00:05.658437
Epoch: [712/1000]	Samples: [184408/259000]	Train Loss: 721.4583584700772	Time: 0:00:05.250595
Epoch: [713/1000]	Samples: [184667/259000]	Train Loss: 714.6997626462959	Time: 0:00:05.302958
Epoch: [714/1000]	Samples: [184926/259000]	Train Loss: 725.7844219428692	Time: 0:00:05.306774
Epoch: [715/1000]	Samples: [185185/259000]	Train Loss: 727.6408851652992	Time: 0:00:05.406651
Epoch: [716/1000]	Samples: [185444/259000]	Train Loss: 727.1248953683036	Time: 0:00:05.323119
Epoch: [717/1000]	Samples: [185703/259000]	Train Loss: 726.480748710485	Time: 0:00:05.165044
Epoch: [718/1000]	Samples: [185962/259000]	Train Loss: 734.4175728462837	Time: 0:00:05.345925
Epoch: [719/1000]	Samples: [186221/259000]	Train Loss: 722.4680378446247	Time: 0:00:05.617500
Epoch: [720/1000]	Samples: [186480/259000]	Train Loss: 724.8663

Epoch: [798/1000]	Samples: [206682/259000]	Train Loss: 725.5025215296212	Time: 0:00:04.790859
Epoch: [799/1000]	Samples: [206941/259000]	Train Loss: 726.2183126206563	Time: 0:00:04.936615
Epoch: [800/1000]	Samples: [207200/259000]	Train Loss: 721.6780826760075	Time: 0:00:05.290313
Epoch: [801/1000]	Samples: [207459/259000]	Train Loss: 718.3060024659146	Time: 0:00:07.629044
Epoch: [802/1000]	Samples: [207718/259000]	Train Loss: 715.7501828698117	Time: 0:00:07.155111
Epoch: [803/1000]	Samples: [207977/259000]	Train Loss: 720.8638194905285	Time: 0:00:09.783854
Epoch: [804/1000]	Samples: [208236/259000]	Train Loss: 717.4467028761462	Time: 0:00:04.986994
Epoch: [805/1000]	Samples: [208495/259000]	Train Loss: 714.7444724300193	Time: 0:00:04.993695
Epoch: [806/1000]	Samples: [208754/259000]	Train Loss: 726.369474315275	Time: 0:00:04.954935
Epoch: [807/1000]	Samples: [209013/259000]	Train Loss: 729.0889793602196	Time: 0:00:04.695680
Epoch: [808/1000]	Samples: [209272/259000]	Train Loss: 727.01

Epoch: [886/1000]	Samples: [229474/259000]	Train Loss: 721.8482114578306	Time: 0:00:08.318350
Epoch: [887/1000]	Samples: [229733/259000]	Train Loss: 722.8644831005671	Time: 0:00:11.989679
Epoch: [888/1000]	Samples: [229992/259000]	Train Loss: 729.2783862964527	Time: 0:00:10.074340
Epoch: [889/1000]	Samples: [230251/259000]	Train Loss: 714.7746073012187	Time: 0:00:05.200138
Epoch: [890/1000]	Samples: [230510/259000]	Train Loss: 720.4403674740589	Time: 0:00:04.926407
Epoch: [891/1000]	Samples: [230769/259000]	Train Loss: 724.7265756967905	Time: 0:00:04.890239
Epoch: [892/1000]	Samples: [231028/259000]	Train Loss: 724.466191707891	Time: 0:00:04.837686
Epoch: [893/1000]	Samples: [231287/259000]	Train Loss: 724.2206098048383	Time: 0:00:06.065225
Epoch: [894/1000]	Samples: [231546/259000]	Train Loss: 721.2141396069619	Time: 0:00:05.084499
Epoch: [895/1000]	Samples: [231805/259000]	Train Loss: 719.0246704572877	Time: 0:00:07.616690


RuntimeError: DataLoader worker (pid 32095) is killed by signal: Terminated. 

In [7]:
ctm.get_topic_lists(5) #[0:5]

/home2/tirthankar/miniconda3_1/envs/rajeev3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['u201cdecipering', 'argument', 'u201cmixed', 'nmt', 'lesser'],
 ['valued', 'complex', 'networks', 'decomposition', 'convacs'],
 ['wide', 'swan', 'beta_i', 'brnn', 'swap'],
 ['theta_t', 'theta_', 'nabla', 'authors', 'think'],
 ['en', 'good', 'bleu', 'total', 'wmt14'],
 ['learning', 'confidence', 'maximin', 'returns', 'case'],
 ['precision', 'three', 'student', 'act', 'predictor'],
 ['state', 'subgraph', 'neighbor', 'adjacency', 'environments'],
 ['network', 'time', 'energy', 'loss', 'using'],
 ['decision', 'universal', 'curved', 'perturbations', 'boundary'],
 ['images', 'np3', 'vaes', 'decoder', 'level'],
 ['singularities', 'layers', 'observations', 'last', 'eq'],
 ['crossover', 'shot', 'physics', 'evolutionary', 'cloning'],
 ['zero', 'distribution', 'lcw', 'activation', 'angle'],
 ['exs', 'major', 'readability', 'dramatically', 'contribute'],
 ['word', 'affect', 'different', 'text', 'nthe'],
 ['starting', 'please', 'variate', 'also', 'stein'],
 ['particular', 'specific', 'model', 'id

In [8]:
import numpy as np

distribution = ctm.get_thetas(training_dataset)[1] # topic distribution for the eight document

print(distribution)

topic = np.argmax(distribution)

ctm.get_topic_lists(10)[topic]

/home2/tirthankar/miniconda3_1/envs/rajeev3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home2/tirthankar/miniconda3_1/envs/rajeev3/lib/python3.7/site-packages/contextualized_topic_models/models/ctm.py:254: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)


[7.10980436e-04 1.62897324e-03 3.98529510e-04 9.48254379e-04
 3.94401594e-04 2.06332653e-04 2.48435562e-03 2.95336597e-03
 5.14506845e-03 1.36826900e-03 4.68691640e-02 2.01619504e-03
 2.28624130e-04 8.89956229e-04 4.14607395e-04 1.52585578e-03
 9.39152134e-04 1.24961914e-03 3.21718086e-04 3.81750645e-04
 1.03304391e-03 6.23064031e-04 1.62250473e-03 1.11184901e-03
 1.64760521e-03 1.21673859e-03 5.63679983e-04 8.40867978e-04
 1.48199542e-03 1.37966381e-04 6.71870853e-04 7.66457272e-04
 3.12873931e-04 4.30998740e-04 5.59187497e-04 9.03200305e-01
 5.91522881e-04 4.92627318e-04 3.88521448e-04 5.38432093e-04
 7.97445071e-04 7.02753582e-04 7.05526935e-04 1.16624888e-03
 4.58874659e-04 1.03766033e-03 8.23515578e-04 4.12564249e-04
 4.14724097e-03 4.40911944e-04]


['task',
 'tasks',
 'novelty',
 'agent',
 'section',
 'known',
 'good',
 'fokker',
 'planck',
 'theory']